In [2]:
import tensorflow as tf

In [3]:
x = tf.Variable(5, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [4]:
sess=tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

106


In [5]:
sess.close()

In [6]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
print(result)

106


In [7]:
init = tf.global_variables_initializer()  #initialises all the variables when it is run, for now it just creates a node

with tf.Session() as sess:
    init.run()  #initialises all the variables now
    result=f.eval()
    print(result)

106


In [8]:
#for interactive session
init = tf.global_variables_initializer() 
sess = tf.InteractiveSession()   #It will be a default session so u don't need a with block to say it is default
init.run()
result = f.eval()
print(result)
sess.close()

106


In [9]:
#learning about graphs
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [10]:
graph = tf.Graph()
with graph.as_default():
    x2=tf.Variable(2)
    
    

In [11]:
x2.graph is graph
x2.graph is tf.get_default_graph()
#x1.graph is tf.get_default_graph()
tf.reset_default_graph()

In [12]:
w = tf.constant(2)
x = w + 2
y = x + 3 
z = x + 8

In [13]:
# this calculates x and w values twice each time for y and z
with tf.Session() as sess: 
    print(y.eval())
    print(z.eval())

7
12


In [14]:
#The following code saves the time by calculating the x and w values only once
with tf.Session() as sess:
    y_val,z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

7
12


In [15]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name = "y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess: 
    theta_value = theta.eval()

In [16]:
print(theta_value)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


In [17]:
from sklearn.preprocessing import StandardScaler 

In [18]:
scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

In [19]:
print(housing.data.shape)
print(housing_data_plus_bias.data.shape)

(20640, 8)
(20640, 9)


In [20]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1,1), dtype=tf.float32, name = "theta")
y_pred = tf.matmul(X,theta, name ="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = 2/m * tf.matmul(tf.transpose(X),error)
training_op = tf.assign(theta, theta-learning_rate*gradients)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            print("epoch", epoch,"MSE", mse.eval())
        sess.run(training_op)
    theta_best= theta.eval()

epoch 0 MSE 9.713478
epoch 100 MSE 4.879631
epoch 200 MSE 4.8328657
epoch 300 MSE 4.824307
epoch 400 MSE 4.8184633
epoch 500 MSE 4.8142505
epoch 600 MSE 4.8112054
epoch 700 MSE 4.809005
epoch 800 MSE 4.8074145
epoch 900 MSE 4.806266


In [21]:
#replace gradients with tf.gradients
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1,1), dtype=tf.float32, name = "theta")
y_pred = tf.matmul(X,theta, name ="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = tf.gradients(mse,[theta])[0]
training_op = tf.assign(theta, theta-learning_rate*gradients)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            print("epoch", epoch,"MSE", mse.eval())
        sess.run(training_op)
    theta_best= theta.eval()

epoch 0 MSE 9.891086
epoch 100 MSE 4.973423
epoch 200 MSE 4.8845863
epoch 300 MSE 4.859814
epoch 400 MSE 4.8440323
epoch 500 MSE 4.8327656
epoch 600 MSE 4.8246336
epoch 700 MSE 4.8187513
epoch 800 MSE 4.8144946
epoch 900 MSE 4.8114142


In [22]:
#replace gradients with optimizer and training_op
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1,1), dtype=tf.float32, name = "theta")
y_pred = tf.matmul(X,theta, name ="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            print("epoch", epoch,"MSE", mse.eval())
        sess.run(training_op)
    theta_best= theta.eval()

epoch 0 MSE 7.7305126
epoch 100 MSE 4.9900994
epoch 200 MSE 4.9079285
epoch 300 MSE 4.877916
epoch 400 MSE 4.8573136
epoch 500 MSE 4.842472
epoch 600 MSE 4.8317304
epoch 700 MSE 4.823947
epoch 800 MSE 4.818313
epoch 900 MSE 4.8142242


In [23]:
#replace gradients with momentum optimizer and training_op
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1,1), dtype=tf.float32, name = "theta")
y_pred = tf.matmul(X,theta, name ="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
optimizer = tf.train.MomentumOptimizer(learning_rate,momentum=0.9)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            print("epoch", epoch,"MSE", mse.eval())
        sess.run(training_op)
    theta_best= theta.eval()

epoch 0 MSE 7.2241073
epoch 100 MSE 4.8117256
epoch 200 MSE 4.803764
epoch 300 MSE 4.8033113
epoch 400 MSE 4.8032618
epoch 500 MSE 4.803255
epoch 600 MSE 4.803253
epoch 700 MSE 4.8032527
epoch 800 MSE 4.8032565
epoch 900 MSE 4.8032546


In [24]:
A = tf.placeholder(dtype=tf.float32, shape=(None,3))
B = A + 5
with tf.Session() as sess : 
    B_value1 = B.eval(feed_dict={A:[[1,2,3]]})
    B_value2 = B.eval(feed_dict={A:[[1,2,3],[4,5,6]]})
print(B_value1)
print(B_value2)

[[6. 7. 8.]]
[[ 6.  7.  8.]
 [ 9. 10. 11.]]
